# 分類アルゴリズムの紹介　1

## 損失関数型アルゴリズム

モデルの構造は数式の関数で決められていて\
パラメータを損失関数が0二近づくように\
調整・最適化していく特徴を持つモデル

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# numpyの浮動小数点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)
# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14
# 乱数の種
random_seed = 123

## 0. サンプルコーディングで用いるデータ

In [ ]:
# サンプルデータの生成

# ライブラリインポート
from sklearn.datasets import make_moons
from sklearn.datasets import make_circles
from sklearn.datasets import make_classification

# 線形分離型
X1, y1 = make_classification(n_features=2, n_redundant=0, 
    n_informative=2, random_state=random_seed, 
    n_clusters_per_class=1, n_samples=200, n_classes=2)

# 三日月型 (線形分離不可)
X2, y2 = make_moons(noise = 0.05, random_state=random_seed, 
    n_samples=200)

# 円形 (線形分離不可)
X3, y3 = make_circles(noise = 0.02, random_state=random_seed, 
    n_samples=200)

# 3種類のデータをDataListに保存
DataList = [(X1, y1), (X2, y2), (X3, y3)]

# N: データの種類数
N = len(DataList)

In [ ]:
# 散布図表示
plt.figure(figsize=(15,4))

# カラーマップ定義
from matplotlib.colors import ListedColormap
cmap = ListedColormap(['#0000FF', '#000000'])

for i, data in enumerate(DataList):
    X, y = data
    ax = plt.subplot(1, N, i+1)
    ax.scatter(X[:,0], X[:,1], c=y, cmap=cmap)
    
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# 決定境界の表示関数
def plot_boundary(ax, x, y, algorithm):
    x_train, x_test, y_train, y_test = train_test_split(x, y,
            test_size=0.5, random_state=random_seed)
    # カラーマップ定義
    from matplotlib.colors import ListedColormap
    cmap1 = plt.cm.bwr
    cmap2 = ListedColormap(['#0000FF', '#000000'])

    h = 0.005
    # アルゴリズムでの学習
    algorithm.fit(x_train, y_train)
    # 学習結果の取得
    score_test = algorithm.score(x_test, y_test)
    score_train = algorithm.score(x_train, y_train)

    # モデルの確信度を表示するための処理
    f1_min = x[:, 0].min() - 0.5
    f1_max = x[:, 0].max() + 0.5
    f2_min = x[:, 1].min() - 0.5
    f2_max = x[:, 1].max() + 0.5
    f1, f2 = np.meshgrid(np.arange(f1_min, f1_max, h), 
                         np.arange(f2_min, f2_max, h))
    # アルゴリズムによって処理を分ける
    # 分類の確信度はdecision_functionとpredict_proba選べるがpredict_probaしかないアルゴリズムがある
    if hasattr(algorithm, "decision_function"):
        # decision_functionはレンジなしの信頼度を表す。
        Z = algorithm.decision_function(np.c_[f1.ravel(), f2.ravel()])
        Z = Z.reshape(f1.shape)
        ax.contour(f1, f2, Z, levels=[0], linewidth=2)
    else:
        # predict_probaはそのクラスに分類される確率を表す。
        Z = algorithm.predict_proba(np.c_[f1.ravel(), f2.ravel()])[:, 1]
        Z = Z.reshape(f1.shape)

    # エリアを表示する
    ax.contourf(f1, f2, Z, cmap=cmap1, alpha=0.3)
    
    # 散布図の表示
    ax.scatter(x_test[:,0], x_test[:,1], c=y_test, cmap=cmap2)
    ax.scatter(x_train[:,0], x_train[:,1], c=y_train, cmap=cmap2, marker='x')
    # 学習結果の表示
    text = f'検証:{score_test:.2f}  訓練: {score_train:.2f}'
    ax.text(f1.max() - 0.3, f2.min() + 0.3, text, horizontalalignment='right',
    fontsize=18) 
    

In [ ]:
# 散布図と決定境界の表示関数

def plot_boundaries(algorithm, DataList):
    plt.figure(figsize=(15,4))
    for i, data in enumerate(DataList):
        X, y = data
        ax = plt.subplot(1, N, i+1)
        plot_boundary(ax, X, y, algorithm)
    plt.show()

# 1. ロジスティック回帰

## ロジスティック回帰は、直線で分類しようとする

### 概要
入力情報を1次関数にかけて結果を出力する\
その結果をシグモイド関数にかけて、0から1の値に変換する\
変換したい値が0.5より大きい場合は１\
変換したい値が0.5より小さい場合は0\
として予測結果とする\

#### シグモイド関数のグラフ表示

In [ ]:
# シグモイド関数の定義
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# xのデータ準備
x = np.linspace(-5, 5, 101)

# yのデータ準備
y = sigmoid(x)

# グラフ表示
plt.plot(x, y, label='シグモイド関数', c='b', lw=2)

# 凡例表示
plt.legend()

# 方眼表示
plt.grid()

# グラフ描画
plt.show()

In [ ]:
# ロジスティック回帰の散布図・分類結果表示

# アルゴリズムの選定
from sklearn.linear_model import LogisticRegression
algorithm = LogisticRegression(random_state=random_seed)

# アルゴリズムの持つパラメータの表示 
print(algorithm)

# 表示関数の呼び出し
plot_boundaries(algorithm, DataList)

## ロジスティック回帰は、直線で分類しようとする


# 2.サポートベクターマシン(カーネル)

## 2次元のデータを3次元に拡張して分類しようとする

### 概要

カーネル法という次元拡張する方法は、\
ガウスカーネル、多項式カーネル、シグモイドカーネルなどがある



ガウスカーネルで、サンプルデータに境界線を引いてみる

In [ ]:
from mpl_toolkits.mplot3d.axes3d import Axes3D

# 2次元乱数を生成
x = np.random.rand(100)
y = np.random.rand(100)
 
# 散布図を描画
# 2次元散布図
plt.scatter(X1[:,0], X1[:,1], c=y1, cmap=cmap)

# 3次元乱数を生成
z = np.random.rand(len(X1[:,0]))

# figureを生成する
fig = plt.figure()
# axをfigureに設定する
ax = Axes3D(fig)

# 散布図を描画
# 3次元散布図
ax.scatter(X1[:,0], z, X1[:,1],  c=y1, cmap=cmap)


In [ ]:
# SVM (カーネル)の散布図・分類結果表示

# アルゴリズムの選定
from sklearn.svm import SVC
algorithm = SVC(kernel='rbf', random_state=random_seed)

# アルゴリズムの持つパラメータの表示 
print(algorithm)

# 表示関数の呼び出し
plot_boundaries(algorithm, DataList)

# 3 ニューラルネットワーク

##　

### 概要




MPLClassifier\
入力層と隠れ層と出力層が全結合である、もっとも単純なディープラーニング\
分類の機械学習をするときに利用される

In [ ]:
# ニューラルネットワークの散布図・分類結果表示
# ノードが1層の場合は、ロジスティック回帰のような境界線の表示をする

# アルゴリズムの選定
from sklearn.neural_network import MLPClassifier
algorithm = MLPClassifier(random_state=random_seed)

# アルゴリズムの持つパラメータの表示 
print(algorithm)

# 表示関数の呼び出し
plot_boundaries(algorithm, DataList)

In [ ]:
# ニューラルネットワークの散布図・分類結果表示
# ノードが100層の場合は、カーネル法のような境界線を表示する

# アルゴリズムの選定
# 隠れ層ノード数=(100,100)
from sklearn.neural_network import MLPClassifier
algorithm = MLPClassifier(hidden_layer_sizes=(100,100), random_state=random_seed)

# アルゴリズムの持つパラメータの表示 
print(algorithm)

# 表示関数の呼び出し
plot_boundaries(algorithm, DataList)

### 参考
バージョン確認用

In [ ]:
import sklearn
print(sklearn.__version__)